In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from tensorflow import keras
from keras.layers.core import Dense
from tensorflow.keras.layers import Input , Flatten
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
pip install split_folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
img_folder='/content/gdrive/MyDrive/sprints project /Scenes training set'
import splitfolders
splitfolders.ratio(img_folder, output="output",
    seed=1337, ratio=(.7,0.3), group_prefix=None, move=False)

Copying files: 14034 files [05:29, 42.63 files/s] 


In [13]:
train_dir = '/content/output/train'
val_dir = '/content/output/val'
test_dir='/content/gdrive/MyDrive/sprints project /testset'

In [6]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen= ImageDataGenerator(rescale=1. /255)
train_generator=train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=30,
    shuffle=False,
    seed=42,
    class_mode='sparse'
)

Found 9820 images belonging to 6 classes.


In [7]:
from keras.preprocessing.image import ImageDataGenerator
val_datagen= ImageDataGenerator(rescale=1. /255)
val_generator= val_datagen.flow_from_directory(
     directory=val_dir,
     target_size=(224, 224),
     color_mode= "rgb",
     batch_size=30,
     shuffle= False,
     seed=42 ,
     class_mode='sparse'
)

Found 4214 images belonging to 6 classes.


In [14]:
from keras.preprocessing.image import ImageDataGenerator
test_datagen= ImageDataGenerator(rescale=1. /255)
test_generator=test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=30,
    shuffle=False,
    seed=42,
   class_mode='sparse'
)

Found 3050 images belonging to 1 classes.


In [15]:
from keras.optimizers import gradient_descent_v2


In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(6, (5, 5), activation='relu', padding='same', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(16, (5, 5), activation='relu', padding='valid'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='relu'),
    tf.keras.layers.Dense(84, activation='relu'),
    
    tf.keras.layers.Dense(6, activation='softmax'),
])

In [18]:
optimiser = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(
    optimizer=optimiser, 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 6)       456       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 6)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 108, 108, 16)      2416      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 16)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 46656)             0         
                                                                 
 dense (Dense)               (None, 120)               5

In [19]:
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=2, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, verbose=1),
]

In [20]:
train_log = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=callbacks
)

Epoch 1/30
328/328 [==============================] - 35s 71ms/step - loss: 1.6648 - accuracy: 0.3073 - val_loss: 1.3324 - val_accuracy: 0.4499 - lr: 0.0010
Epoch 2/30
328/328 [==============================] - 24s 72ms/step - loss: 1.2038 - accuracy: 0.4829 - val_loss: 1.0633 - val_accuracy: 0.5384 - lr: 0.0010
Epoch 3/30
328/328 [==============================] - 24s 72ms/step - loss: 1.0084 - accuracy: 0.5747 - val_loss: 0.9836 - val_accuracy: 0.5997 - lr: 0.0010
Epoch 4/30
328/328 [==============================] - 23s 71ms/step - loss: 0.8508 - accuracy: 0.6573 - val_loss: 1.0084 - val_accuracy: 0.5733 - lr: 0.0010
Epoch 5/30
328/328 [==============================] - 23s 71ms/step - loss: 0.6762 - accuracy: 0.7307 - val_loss: 0.8764 - val_accuracy: 0.6630 - lr: 0.0010
Epoch 6/30
328/328 [==============================] - 23s 71ms/step - loss: 0.4405 - accuracy: 0.8337 - val_loss: 1.0573 - val_accuracy: 0.6111 - lr: 0.0010
Epoch 7/30
328/328 [==============================] - 24s 

In [22]:
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size
model.evaluate_generator(generator=val_generator,
steps=STEP_SIZE_VALID)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


[1.8650150299072266, 0.6757143139839172]

In [23]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
model.evaluate_generator(generator=train_generator,
steps=STEP_SIZE_TRAIN)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


[0.005156552419066429, 0.9990825653076172]

In [26]:
import os 
import glob
import pandas as pd

Name = []

for img in glob.glob(test_dir):
    prediction = model.predict_generator(test_generator,steps=150,verbose=1)
    for name in os.listdir('/content/gdrive/MyDrive/sprints project /testset/Scenes testing test'):
         Name.append(name[0::])



predicted_class_indics=np.argmax(prediction,axis=1)
Name.sort()
Temp = {'Image':Name, 'Label':predicted_class_indics}
temp = pd.DataFrame(Temp)
temp.to_csv('/content/results.csv', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  


102/150 [===================>..........] - ETA: 4s

150/150 [==============================] - 10s 68ms/step
